In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [10]:
from talus_data_analysis.s3 import *
from talus_data_analysis.plot import histogram
from talus_data_analysis.elib import Elib
from dotenv import load_dotenv
import tempfile
import sqlite3
import pandas as pd

In [11]:
load_dotenv()

True

In [12]:
ENCYCLOPEDIA_BUCKET = "talus-data-pipeline-encyclopedia-bucket"
PROTEINS_KEY = "wide/210308_MM1S/RESULTS-quant.elib.proteins.txt"
ELIB_FILE="wide/210308_MM1S/210308_talus_13.mzML.elib"

In [13]:
elib_files = get_file_keys_in_bucket(bucket=ENCYCLOPEDIA_BUCKET, key="narrow/210308_MM1S", file_type=".elib")

In [14]:
elib_files

['narrow/210308_MM1S/210308_talus_16.mzML.elib',
 'narrow/210308_MM1S/210308_talus_17.mzML.elib',
 'narrow/210308_MM1S/210308_talus_18.mzML.elib',
 'narrow/210308_MM1S/210308_talus_19.mzML.elib',
 'narrow/210308_MM1S/210308_talus_20.mzML.elib',
 'narrow/210308_MM1S/210308_talus_21.mzML.elib',
 'narrow/210308_MM1S/RESULTS-chr.elib']

In [17]:
def number_of_peptides_per_file(elib_file: str):
    sql = """SELECT SourceFile, COUNT(SourceFile) 
    FROM peptidescores
    WHERE isDecoy == 0
    GROUP BY SourceFile;"""

    peptides_per_file = {}

    elib_conn = Elib(key=elib_file, bucket=ENCYCLOPEDIA_BUCKET)
    peptides_per_file = {file: peptides for (file, peptides) in elib_conn.execute_sql(sql)}
    elib_conn.close()

    return peptides_per_file

In [18]:
peptides_per_file = {}
for elib_file in elib_files:
    print(elib_file)
    if not "RESULTS" in elib_file:
        peptides_per_file.update(number_of_peptides_per_file(elib_file=elib_file))

narrow/210308_MM1S/210308_talus_16.mzML.elib


KeyboardInterrupt: 

In [19]:
peptides_per_file

{}

In [20]:
peps = [{"File": key, "# peptides": value, "Type": "narrow" if f"{key}.elib" in set([os.path.basename(f) for f in elib_files]) else "wide"} for key, value in peptides_per_file.items()]

In [ ]:
df = pd.DataFrame(peps).sort_values(by="# peptides", ascending=False).reset_index(drop=True)

In [ ]:
df.to_csv("~/Downloads/210308_MM1S_Peptides.csv")

In [ ]:
fig = histogram(df=df,
                x_label_column_name="File",
                y_label_column_name="# peptides",
                color="Type",
                xaxis_title="# of Peptides",
                yaxis_title="Number of Proteins")

fig.show()

In [22]:
elib = "wide/210308_MM1S/210308_talus_13.mzML.elib"

elib_conn = Elib(key=elib, bucket=ENCYCLOPEDIA_BUCKET)

In [23]:
sql = """SELECT p.ProteinAccession, COUNT(pep.PeptideModSeq) as numPeptides
FROM peptidequants pep
JOIN peptidescores s using (peptidemodseq, precursorcharge)
LEFT JOIN peptidetoprotein p
WHERE pep.PeptideSeq = p.PeptideSeq
GROUP BY p.ProteinAccession
ORDER BY 2 DESC"""

In [24]:
df = pd.DataFrame(elib_conn.execute_sql(sql), columns=["ProteinName", "NumPeptides"])
df["ProteinName"] = df["ProteinName"].apply(lambda x: x.split("|")[-1].replace("_HUMAN", ""))

In [ ]:
df.to_csv("~/Downloads/210308_MM1S_13.csv")

In [ ]:
fig = histogram(df=df,
                x_label_column_name="NumPeptides",
                xaxis_title="# of Peptides",
                yaxis_title="Number of Proteins",
                value_cutoff=30)

fig.show()

In [25]:
df

,ProteinName,NumPeptides
0,PRKDC,238
1,PLEC,237
2,SPTN1,174
3,DYHC1,174
4,MACF1,157
...,...,...
7973,NCOA3,1
7974,Z780B,1
7975,SETBP,1
7976,NS1BP,1
